<a href="https://colab.research.google.com/github/EPC-MSU/biquad_filters_comparison/blob/master/test_biquad_collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Установка зависмостей
print()
print(' PIP INSTALL '.center(80, '='))
print()
!pip3 install control
!pip3 install tabulate
!pip3 install ipympl
print()
print(' COPY DEPENDENCIES FROM GIT '.center(80, '='))
print()
!wget https://raw.githubusercontent.com/EPC-MSU/biquad_filters_comparison/refs/heads/master/fixpt.py -O fixpt.py
!wget https://raw.githubusercontent.com/EPC-MSU/biquad_filters_comparison/refs/heads/master/biquad_filters.py -O biquad_filters.py

In [ ]:
# @title Инициализация
# @markdown **Интерактивные графики** (Zoom для результатов моделирования, сохранение и т.п.)
# @markdown
# @markdown Работают в Collab нестабильно, ниже процедура включения:
# @markdown * Провети инициализацию без поддержки `INTERACTIVE_PLOT`.
# @markdown * Перезапустить сеанс (меню "Среда выполнения").
# @markdown * Провети инициализацию с включенной поддержкой `INTERACTIVE_PLOT`.
INTERACTIVE_PLOT = False # @param {type:"boolean"}

import scipy.signal as sig
import control as ctrl
import numpy as np

if INTERACTIVE_PLOT:
  # enable widget backend
  from google.colab import output
  output.enable_custom_widget_manager()
  # use widget backend
  %matplotlib widget
  import matplotlib.pyplot as plt
  # figures width
  FIGURE_WIDTH = 16
else:
  # use inline backend
  %matplotlib inline
  import matplotlib.pyplot as plt
  FIGURE_WIDTH = 18

import fixpt
from biquad_filters import BiQuad, BiQuadDF1, BiQuadCoupled

# Процедуры синтеза фильтров
def lowpass1(f_cut, T):
    ''' Lowpass 1nd order butterwort filter synthesis.
    '''
    a = np.exp(-2*np.pi*f_cut*T)
    return [1-a, 0, 0], [1, -a, 0]

def lowpass2(f_cut, T):
    ''' Lowpass 2nd order butterwort filter synthesis.
    '''
    f_nyquist = 0.5/T
    b, a = sig.butter(2, f_cut/f_nyquist)
    return b, a

def notch2(f_nominal, f_width, T):
    ''' Two parameters Notch (band-stop) filter synthesis.

        Calculate coffecients of notch filter with stop-band center at f_nominal (Hz) with width f_width (Hz).
        T is discretization period (s). Returns numerator and denominatror (b, a) of transfer function.
    '''
    wn = 2.0*np.pi * f_nominal * T
    wc = 2.0*np.pi * f_width * T
    a = 1.0 / np.cos(wc / 2.0) - np.tan(wc / 2.0)
    K = (1.0 - 2.0*a*np.cos(wn) + a**2) / (2.0 - 2.0*np.cos(wn))
    b = [K, -2.0*K*np.cos(wn), K]
    a = [1.0, -2.0*a*np.cos(wn), a**2]
    return b, a

def notch3(f_nominal, f_width, L_stop, T):
    ''' Notch (band-stop) filter synthesis.

        Calculate coffecients of notch filter with stop-band center at f_nominal (Hz) with width f_width (Hz).
        L_stop (dB) is filter attenuation at f_nominal frequency. It must be lower then zero.
        T is discretization period (s). Returns numerator and denominatror (b, a) of transfer function.
    '''
    # check parameters
    if T <= 0.0:
        raise ValueError('T must be posititve')
    f_nyquist = 1/(2*T)
    if f_nominal < 0.0 or f_nominal > f_nyquist:
        raise ValueError('f_nominal must be in interval [0.0, 1/(2*T)]')
    if f_width < 0.0 or f_width >= 2*f_nominal:
        raise ValueError('f_width must be in interval [0.0, 2*f_nominal]')
    if L_stop >= 0.0:
        raise ValueError('L_stop must be negative.')
    # convert Hz to rad/s
    Wn = 2*np.pi*f_nominal
    Wc = 2*np.pi*f_width
    # notch parameters
    b = 10**(L_stop/20)
    p = (Wn - Wc/2)/Wn
    a = 0.5 * np.sqrt( (p - 1/p)**2 / (1 - 2*b**2) )
    # tf numerator and denominator
    den = np.array([1, - 2*np.exp(-a*Wn*T)*np.cos(Wn*T*np.sqrt(1-a**2)), np.exp(-2*a*Wn*T)])
    num = np.array([1, - 2*np.exp(-a*b*Wn*T)*np.cos(Wn*T*np.sqrt(1-a**2*b**2)), np.exp(-2*a*b*Wn*T)])
    num *= np.sum(den) / np.sum(num)
    return num, den

In [ ]:
# @title Параметры IP-блоков (HWH)
# период дискретизации (с)
# @markdown ### Общие парметры реализации
BASE_FREQ = 20000 # @param {type:"integer", min: 1}
SIGNAL_NBITS = 18 # @param {type:"integer", min: 1}
T = 1/BASE_FREQ
# @markdown ### Параметры реализации блока DF1
A_NBITS = 18   # @param {type:"integer", min: 1, max: 32}
A1_FRACBITS = 16   # @param {type:"integer", min: 1, max: 32}
A2_FRACBITS = 17   # @param {type:"integer", min: 1, max: 32}
B_NBITS = 27 # @param {type:"integer", min: 1, max: 32}
B_FRACBITS = 24 # @param {type:"integer", min: 1, max: 32}
STATE_NBITS = 27 # @param {type:"integer", min: 1, max: 64}
STATE_FRACBITS = 9 # @param {type:"integer", min: 1, max: 64}
QUANT_POLICY = fixpt.QuantPolicy.TruncateToZero # @param [ "fixpt.QuantPolicy.Truncate", "fixpt.QuantPolicy.TruncateToZero", "fixpt.QuantPolicy.Round" ] {type: 'raw'}
df1_config = BiQuadDF1.Config(BASE_FREQ=BASE_FREQ, SIGNAL_NBITS=SIGNAL_NBITS, A_NBITS=A_NBITS, A1_FRACBITS=A1_FRACBITS, A2_FRACBITS=A2_FRACBITS, B_NBITS=B_NBITS, B_FRACBITS=B_FRACBITS, STATE_NBITS=STATE_NBITS, STATE_FRACBITS=STATE_FRACBITS, QUANT_POLICY=QUANT_POLICY)
# @markdown ### Параметры реализации блока Coupled
ALPHA_NBITS = 18 # @param {type:"integer", min: 1, max: 32}
ALPHA_FRACBITS = 17 # @param {type:"integer", min: 1, max: 32}
Q_NBITS = 27 # @param {type:"integer", min: 1, max: 32}
Q_FRACBITS = 25 # @param {type:"integer", min: 1, max: 32}
STATE_NBITS = 27 # @param {type:"integer", min: 1, max: 64}
STATE_FRACBITS = 9  # @param {type:"integer", min: 1, max: 64}
QUANT_POLICY = fixpt.QuantPolicy.TruncateToZero # @param [ "fixpt.QuantPolicy.Truncate", "fixpt.QuantPolicy.TruncateToZero", "fixpt.QuantPolicy.Round" ] {type: 'raw'}
coup_config = BiQuadCoupled.Config(BASE_FREQ = BASE_FREQ, SIGNAL_NBITS=SIGNAL_NBITS, ALPHA_NBITS=ALPHA_NBITS, ALPHA_FRACBITS=ALPHA_FRACBITS, Q_NBITS=Q_NBITS, Q_FRACBITS=Q_FRACBITS, STATE_NBITS=STATE_NBITS, STATE_FRACBITS=STATE_FRACBITS, QUANT_POLICY=QUANT_POLICY)

In [ ]:
# @title Задание фильтра
filter_type = "notch" # @param ['lowpass1', 'lowpass2', 'highpass1', 'highpass2', 'notch', 'notch_inf', 'bandpass', 'bandstop'] {type:"string"}
# @markdown **Параметры lowpass и highpass фильтров**: частота среза `f_cut` (Гц)
f_cut = 50 # @param {type:"number", min: 0.0}
# @markdown **Параметры notch фильтров**: номинальная частота `f_nominal` (Гц), ширина полосы подавления `f_width` (Гц), подавление сигнала на номинаьной частоте `L_stop` (Дб)
f_nominal = 50 # @param {type:"number", min: 0.0}
f_width = 5 # @param {type:"number", min: 0.0}
L_stop = -100 # @param {type:"number", max: 0.0}
# @markdown **Параметры bandstop и bandpass фильтров**: начало `f_start` и конец `f_end` полосы подавления/пропускания (Гц)
f_start = 50  # @param {type:"number", max: 0.0}
f_end = 5000 # @param {type:"number", max: 0.0}

# расчет коэффициентов фильтров
f_nyquist = 1 / (2*T)
if filter_type == 'lowpass1':
    b, a = lowpass1(f_nominal, T)
elif filter_type == 'lowpass2':
    b, a = lowpass2(f_nominal, T)
elif filter_type == 'highpass1':
    b, a = lowpass1(f_nominal, T)
    b = np.array(a) - np.array(b)
elif filter_type == 'highpass2':
    b, a = lowpass2(f_nominal, T)
    b = np.array(a) - np.array(b)
elif filter_type in ('bandpass', 'bandstop'):
    b, a = sig.butter(1, [f_start/f_nyquist, f_end/f_nyquist], filter)
elif filter_type == 'notch':
    b, a = notch3(f_nominal, f_width, L_stop, T)
elif filter_type == 'notch_inf':
    b, a = notch2(f_nominal, f_width, T)
else:
    raise ValueError('unknown filter type')

# передаточные функции
Hz = BiQuad(b, a, T)
Hz.name = filter_type
# DF1
regs_df1 = BiQuadDF1.Registers(df1_config).from_tf(b, a)
Hz_df1 = BiQuadDF1(regs_df1, df1_config)
# Сoupled
regs_coup = BiQuadCoupled.Registers(coup_config)
try:
    regs_coup.from_tf(b, a)
except ValueError:
    pass
Hz_coup = BiQuadCoupled(regs_coup, coup_config)

# результаты
print(Hz)
print()
print(regs_df1)
print()
print(regs_coup)

# диаграммы Боде
fig = plt.figure(figsize=(14,8))
out = ctrl.bode_plot([Hz, Hz_df1, Hz_coup], Hz = True, dB = True, label=('float', 'DF1', 'Coupled'), omega=(2*np.pi, np.pi/T-1), title='Диаграмма Боде фильтра и его реализации с учетом квантования коэффициентов', figure=fig)

In [ ]:
# @title Моделирование работы фильтра
# @markdown #### Тестовый сигнал
signal_type = 'saw' # @param ['chirp', 'sine', 'saw', 'square'] {type: 'string'}
duration = 2.0 # @param {type: 'number', min: 0}
amplitude = 100 # @param {type: 'integer', min: 0}
frequency = 10.0 # @param {type: 'number', min: 0}
chirp_start_frequency = 1.0 # @param {type: 'number', min: 0}
chirp_end_frequency = 100.0 # @param {type: 'number', min: 0}

t = np.arange(0.0, 1.1*duration, T)
if signal_type == 'chirp':
  u = amplitude * sig.chirp(t, chirp_start_frequency, duration, chirp_end_frequency, method='linear')
elif signal_type == 'sine':
  u = amplitude * np.sin(2*np.pi*frequency*t)
elif signal_type == 'saw':
  u = amplitude * sig.sawtooth(2*np.pi*frequency*t, 0.5)
elif signal_type == 'square':
  u = amplitude * sig.square(2*np.pi*frequency*t, 0.5)
else:
  raise ValueError('Unknown signal type.')
u[t > duration] = 0.0

# моделирование
_, y = Hz.output(u)
_, y_df1 = Hz_df1.output(u)
_, y_coup = Hz_coup.output(u)

# построение графиков
fig, axs = plt.subplots(3, 1, figsize=(FIGURE_WIDTH,18), sharex=True)
axs[0].plot(t, u)
axs[0].set_xlabel('t, s')
axs[0].set_ylabel('u')
axs[0].set_title('Входной сигнал')
axs[0].grid()
axs[1].plot(t, y, label='float')
axs[1].plot(t, y_df1, label='DF1')
axs[1].plot(t, y_coup, label='Coupled')
axs[1].set_xlabel('t, s')
axs[1].set_ylabel('y')
axs[1].legend()
axs[1].grid()
axs[1].set_title('Сранвение выходов реализаций фильтров')
axs[2].plot(t, y - y_df1, label='DF1 error')
axs[2].plot(t, y - y_coup, label='Coupled error')
axs[2].set_xlabel('t, s')
axs[2].set_ylabel('e')
axs[2].legend()
axs[2].grid()
axs[2].set_title('Расхождение с реализацие на плавающих точках')

plt.show()